In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# --- Custom Function for MAPE ---
# We use the same function as before for a consistent comparison
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

# --- 1. Load and Prepare the Data ---
df = pd.read_excel('Final_Without_IV_2.xlsx')
X = df.drop('Option_Price_C', axis=1)
y = df['Option_Price_C']

# --- 2. Split and Scale the Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Data has been split and scaled.")

# --- 3. Build and Compile the Model ---
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Add more metrics to monitor during training
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[
        tf.keras.metrics.RootMeanSquaredError(),
        tf.keras.metrics.MeanAbsoluteError(),
        tf.keras.metrics.MeanAbsolutePercentageError()
    ]
)

# Learning rate scheduler callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.000005,
    verbose=1
)

model.summary()

# --- 4. Train the Model ---
print("\nStarting model training...")
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    validation_split=0.2,
    callbacks=[lr_scheduler],
    verbose=1
)
print("Model training finished.")

# --- 5. Evaluate Model and Calculate All Metrics ---
print("\n--- Evaluating Neural Network Performance on Test Set ---")

# Make predictions on the test set
predictions = model.predict(X_test_scaled).flatten()

# Calculate the metrics
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
mape = mean_absolute_percentage_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

# Print the comprehensive results
print(f"R-Squared (R²):                  {r2:.4f}")
print(f"Mean Absolute Error (MAE):     {mae:,.2f}")
print(f"Mean Abs. Percentage Err (MAPE): {mape:.2f}%")
print(f"Mean Squared Error (MSE):      {mse:,.2f}")
print(f"Root Mean Squared Error (RMSE):  {rmse:,.2f}")
print("----------------------------------------------------------\n")


# --- 6. Show Sample Predictions ---
results = pd.DataFrame({'Actual_Price': y_test, 'Predicted_Price': predictions})
print("Sample Predictions:")
# Display more rows to get a better feel for the performance
print(results.head(20))

Data has been split and scaled.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,649 (45.50 KB)

 Trainable params: 11,649 (45.50 KB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/100
3320/3320 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 294856.1875 - mean_absolute_error: 234.7432 - mean_absolute_percentage_error: 2051.3237 - root_mean_squared_error: 504.8907 - val_loss: 1852.2152 - val_mean_absolute_error: 27.4550 - val_mean_absolute_percentage_error: 92.3250 - val_root_mean_squared_error: 43.0374 - learning_rate: 0.0010
Epoch 2/100
3320/3320 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 1825.9365 - mean_absolute_error: 26.5813 - mean_absolute_percentage_error: 77.1057 - root_mean_squared_error: 42.7292 - val_loss: 1534.5267 - val_mean_absolute_error: 24.5411 - val_mean_absolute_percentage_error: 64.1947 - val_root_mean_squared_error: 39.1730 - learning_rate: 0.0010
Epoch 3/100
3320/3320 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 1662.4933 - mean_absolute_error: 25.0371 - mean_absolute_percentage_error: 68.5168 - root_mean_squared_error: 40.7728 - val_loss: 1505.9500 - val_mean_absolute_error: 23.5070 - val_mean_absolute_percentage_